In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

# Preprocessing

In [3]:
all_text=newdata["transcription_text"]

In [4]:
all_text[0]

'[M177 ENCL]\r\n[[pre-printed]] I, [[handwritten]] I C Meekins [[/handwritten]], of [[handwritten]] Currituck [[/handwritten]] County, State of North Carolina, do solemnly swear or affirm, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the Union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves.  So help me God. \r\n\r\n[[handwritten]] Isaac C. Meekins [[/handwritten]]\r\n\r\nSworn and subscribed to this the [[handwritten]] 16 [[/handwritten]] day of [[handwritten]] Sept [[/handwritten]] A.D., 1865, before \r\n[[handwritten]] E H Walker [[/handwritten]], J.P.\r\nIt is hereby certified that the above is a true copy of the original oath taken and subscribed by [[handwritten]] I C Meekins\r\nE H Walker, J.P.\r\nJohn B Etheridge, J.P.\r\n[[\\ha

In [5]:
all_text[0]

'[M177 ENCL]\r\n[[pre-printed]] I, [[handwritten]] I C Meekins [[/handwritten]], of [[handwritten]] Currituck [[/handwritten]] County, State of North Carolina, do solemnly swear or affirm, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the Union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves.  So help me God. \r\n\r\n[[handwritten]] Isaac C. Meekins [[/handwritten]]\r\n\r\nSworn and subscribed to this the [[handwritten]] 16 [[/handwritten]] day of [[handwritten]] Sept [[/handwritten]] A.D., 1865, before \r\n[[handwritten]] E H Walker [[/handwritten]], J.P.\r\nIt is hereby certified that the above is a true copy of the original oath taken and subscribed by [[handwritten]] I C Meekins\r\nE H Walker, J.P.\r\nJohn B Etheridge, J.P.\r\n[[\\ha

In [6]:
all_text[1]

'^[[ [M177 ENCL] ]]\r\nDepartment of Virginia and North Carolina, ^[[Feb 23]]\r\n^[[Newbern Dec 30th]] 1864.\r\n^[[No 1735]]\r\n^[[I. C. [[Meekins]] ]] has taken and subscribed the OATH and PAROLE required by  General Orders No. 49, for a Loyal Citizen of the United States, residing in a State ever in rebellion, or who has sympathized with the seceding States. \r\n^[[W. C. Hunt]]\r\n^[[Lt & Dept Pro Marshall]]'

In [7]:
all_text[2]

'[P 64 ENCL]\r\n[[Preprinted]]I,[[/Preprinted]] W. H. Pearce, [[Preprinted]]of[[/Preprinted]] Craven [[Preprinted]]County, State of North-Carolina, do solemnly swear or affirm, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of\r\nthe United States and the Union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. So help me God.[[/Preprinted]]\r\n\r\nW. H. Pearce\r\n\r\n[[Preprinted]]Sworn and subscribed to this[[/Preprinted]] 3rd [[Preprinted]]day of[[/Preprinted]] October,\r\n[[Preprinted]]A.D., 1865, before[[/Preprinted]]\r\nJm. E. Ampetts[?] [[Preprinted]], J. P.\r\nIt is hereby certified that the above is a true copy of the original oath taken and subscribed by[[/Preprinted]] W. H. Pearce.\r\n\r\nJm. E. Ampetts[?], J.P.\r\nA. B. Watson[?], J.P.\r\n______________

In [8]:
all_text[3]

'State of North Carolina,}\r\n________ County.                       }\r\nI, ________, do solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought nor accepted, nor attempted to exercise the functions of any office whatever under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power or constitution within the United States, hostile or inimical thereto. And I do further swear, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign, and domestic; that I will bear true faith and allegiance to the same, while I hold affair thirein that I take this obligation freely, without any mental rese

In [9]:
all_text[4]

'State of North Carolina, \r\nWake]] County. \r\n\r\nI, S.E. Allen, do solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought nor accepted, nor attempted to exercise the functions of any office whatever under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power or constitution within the United States, hostile or inimical thereto. And I do further swear, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign and domestic; that I will bear true faith and allegiance to the same; that I take this obligation freely, without any mental reservation or purpose of evasion; and that I will w

In [10]:
all_text[5]

'I Sarah E Dough, of Currituck county, State of North  Carolina do solemnly swear or affirm in presence of Almighty God, that I will hence forth faithfully support, protect and defend the Constitution of the United States and the Union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slave. So help me God\r\n\r\nSarah. E. Dough\r\nsworn and subscribed to this the 1^st day of {Dec.\r\nA.D. 1866, before\r\n\r\nIsaac C. Merkins, J.P.'

In [11]:
all_text[100]  

'[[preprinted]]\r\nState of North Carolina, }\r\n^[[Randolph]] County. }\r\n\r\nI, ^[[W R Frazier]], do solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought nor accepted, nor attempted to exercise the functions of any office whatever under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power or constitution within the United States, hostile or inimical thereto. And I do further swear, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign and domestic; that I will bear true faith and allegiance to the same; that I take this obligation freely, without any mental reservation or purp

In [12]:
all_text[100]

'[[preprinted]]\r\nState of North Carolina, }\r\n^[[Randolph]] County. }\r\n\r\nI, ^[[W R Frazier]], do solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought nor accepted, nor attempted to exercise the functions of any office whatever under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power or constitution within the United States, hostile or inimical thereto. And I do further swear, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign and domestic; that I will bear true faith and allegiance to the same; that I take this obligation freely, without any mental reservation or purp

In [13]:
all_text[100]

'[[preprinted]]\r\nState of North Carolina, }\r\n^[[Randolph]] County. }\r\n\r\nI, ^[[W R Frazier]], do solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought nor accepted, nor attempted to exercise the functions of any office whatever under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power or constitution within the United States, hostile or inimical thereto. And I do further swear, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign and domestic; that I will bear true faith and allegiance to the same; that I take this obligation freely, without any mental reservation or purp

In [14]:
all_text[1200]

'Copy  G6\r\nI, James L. Gantt solemnly swear that I have carefully read the Amnesty Proclamation issued by Andrew Johnson, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made.  /signed/ Jas L. Gantt\r\nSworn to and subscribed before me at Charleston, S.C. this 31 day of August 1865.\r\n/signed/ Levi Stuber\r\nMaj 47 Pa Vols\r\nPro Marshal\r\n\r\nNo 992\r\nUnites States of America\r\nI, James L Gantt of the City of Charleston, State of South Carolina do solemnly swear in the presence of Almighty God that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves, so help me God. / signed/Jas L. Gantt\r\nSworn t

In [15]:
all_text[1200]

'Copy  G6\r\nI, James L. Gantt solemnly swear that I have carefully read the Amnesty Proclamation issued by Andrew Johnson, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made.  /signed/ Jas L. Gantt\r\nSworn to and subscribed before me at Charleston, S.C. this 31 day of August 1865.\r\n/signed/ Levi Stuber\r\nMaj 47 Pa Vols\r\nPro Marshal\r\n\r\nNo 992\r\nUnites States of America\r\nI, James L Gantt of the City of Charleston, State of South Carolina do solemnly swear in the presence of Almighty God that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves, so help me God. / signed/Jas L. Gantt\r\nSworn t

In [16]:
all_text[1203]

'against the said property, but was unable to do so by reason of the absence of the Tax collector. Your petitioner therefore requests that an order be granted to her to enable her to regain possession of her premises aforesaid. (Known as No. 58 Cannon Street and held by Richard Roper as Trustee for the benefit of herself and daughters.) so that she can continue to realize an income by renting the same of which she is greatly in need.\r\n\r\nAs an evidence of loyalty your petitioner has taken the oath of allegiance, a copy of which is herein enclosed.\r\n\r\nAnd your Petitioner will ever pray: and so forth. \r\n\r\nAnne W. Gibbes\r\n\r\nCheraw So. Ca.\r\nSeptember 11th 1865\r\n\r\n\r\nDuplicate\r\n\r\n[Copy.]  \r\nHEADQUARTERS UNITED STATES FORCES.\r\nNO. 604 PROVOST MARSHAL\'S OFFICE.\r\nCheraw S.C. 11th Sept 1865.\r\n\r\nI A.W. Gibs, do solemnly sear, in presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States and the Union o

In [17]:
all_text[1201]

'I, P.T. Gervais solemnly swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, [[strikethrough]] except [[/strikethrough]]\r\n\r\nPaul T. Gervais\r\n\r\nSworn to and subscribed before me at Charleston SC this 1st day of oct 1865.\r\nJ Miller Thompson\r\nCp & 33 U.S.C.T. & P. M.\r\nNo. 1621\r\n\r\nUnited States of America\r\n\r\nPT Garvais [[Gervais]] of the [[strikethrough]] County [[/strikethrough]] Dist of Charleston State of South CA do solemnly swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder that I will, in like manner, abid by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emanci

In [18]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [19]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# without punctuation

In [20]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [21]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])

100%|██████████| 2136/2136 [00:08<00:00, 249.20it/s]


In [22]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:00<00:00, 3264.61it/s]


In [23]:
all_text

0       ] I , ] I C Meekins ] , of ] Currituck ] Count...
1       ^ ] ] Department of Virginia North Carolina , ...
2       ] I , ] W. H. Pearce , ] of ] Craven ] County ...
3       State of North Carolina , } ________ County . ...
4       State of North Carolina , Wake ] ] County . I ...
                              ...                        
2131    Office Provost Marshal Raleigh , N.C. May 26 '...
2132    July 30th 1867 Birnie Wm Agent Bu R F & A L Oa...
2133    State of North Carolina , } Robeson County . I...
2134    NARA 449 102 We whose name written well person...
2135    NARA 450 404 Office Provost Marshall City of W...
Name: transcription_text, Length: 2136, dtype: object

In [24]:
all_text[0]

'] I , ] I C Meekins ] , of ] Currituck ] County , State of North Carolina , solemnly swear affirm , presence of Almighty God , I henceforth faithfully support , protect defend Constitution of United States Union of States thereunder ; I , like manner , abide faithfully support law proclamation made existing rebellion reference emancipation of slave . So help God. ] Isaac C. Meekins ] Sworn subscribed ] 16 ] day of ] Sept ] A.D. , 1865 , ] E H Walker ] , J.P . It hereby certified true copy of original oath taken subscribed ] I C Meekins E H Walker , J.P. John B Etheridge , J.P. ] ]'

# Flair

In [25]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 35.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Hugging face  : Flair

**ner-english** 

In [26]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.22 MiB/s, done.


In [27]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-09 18:28:37,165 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-09 18:29:04,873 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [28]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-09 18:29:15,073 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-09 18:29:16,930 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [29]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-09 18:30:08,748 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-09 18:30:26,480 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


tagger3 nerenglishlarge

In [30]:
listenglarge=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        dflarge.loc[i,"text"]=all_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listenglarge.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG                        

In [31]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,"] I , ] I C Meekins ] , of ] Currituck ] Count...","[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck, State of North Carolina, United St...",[Constitution]
1,"^ ] ] Department of Virginia North Carolina , ...",[],"[North Carolina, United States, States]",[OATH PAROLE]
2,"] I , ] W. H. Pearce , ] of ] Craven ] County ...","[W . H. Pearce, Almighty, God, Jm . E. Ampetts...","[Craven, County, State of North-Carolina, Unit...",[]
3,"State of North Carolina , } ________ County . ...",[God],"[State of North Carolina, United States]",[Constitution]
4,"State of North Carolina , Wake ] ] County . I ...","[S.E, Allen, God, T.D, McAlpine]","[State of North Carolina, Wake, County, United...","[Constitution, Res . Corp]"
...,...,...,...,...
2131,"Office Provost Marshal Raleigh , N.C. May 26 '...","[Geo . B. Ayer, Hugo Hillebrandt]","[Raleigh, N.C., Wake County, United States, No...",[]
2132,July 30th 1867 Birnie Wm Agent Bu R F & A L Oa...,[],[],[Birnie Wm]
2133,"State of North Carolina , } Robeson County . I...","[William Birnie, God, Wm Birnie, Dick J.P]","[State of North Carolina, Robeson County, Unit...",[Constitution]
2134,NARA 449 102 We whose name written well person...,"[Price, James Fulton, Sgd W . N. Peden, Miles ...","[State of North Carolina, New Hanover County, ...","[Fulton & Price, Confederate Government, Court..."


tagger1 ner english

In [32]:
listeng=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dfner=pd.DataFrame({"text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        dfner.loc[i,"text"]=all_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listeng.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dfner["PERSON-nerenglish"]=Per
dfner["LOC-nerenglish"]=LOC
dfner["ORG-nerenglish"]=ORG                        

In [33]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,"] I , ] I C Meekins ] , of ] Currituck ] Count...","[God, Isaac C. Meekins, I C Meekins E H Walker...",[Currituck ] County],[]
1,"^ ] ] Department of Virginia North Carolina , ...",[],[United States],[]
2,"] I , ] W. H. Pearce , ] of ] Craven ] County ...","[H. Pearce, God, E. Ampetts, B. Watson]","[Craven ] County, State of North-Carolina]",[]
3,"State of North Carolina , } ________ County . ...","[God, Sworn]",[United States],[]
4,"State of North Carolina , Wake ] ] County . I ...","[Allen, God, Allen Sworn]",[United States],[]


tagger 2 ontonotes

In [34]:
listoto=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

aldf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        aldf.loc[i,"text"]=all_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listoto.append(entity)
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
aldf["Date-ontonotes"]=Date
aldf["PERSON-ontonotes"]=Per
aldf["GPE-ontonotes"]=GPE
aldf["ORG-ontonotes"]=ORG    

In [35]:
aldf.shape

(2136, 5)

In [36]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,"] I , ] I C Meekins ] , of ] Currituck ] Count...","[I C Meekins, Isaac C. Meekins, E H Walker]",[North Carolina],[],"[16 ] day of ] Sept ] A.D . , 1865]"
1,"^ ] ] Department of Virginia North Carolina , ...",[],"[North Carolina, United States]",[Department of Virginia],[1864]
2,"] I , ] W. H. Pearce , ] of ] Craven ] County ...",[W . H. Pearce],[],[],"[3rd ] day of ] October , ] A.D . , 1865]"
3,"State of North Carolina , } ________ County . ...",[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]"
4,"State of North Carolina , Wake ] ] County . I ...",[S.E . Allen],"[North Carolina, United States]",[],"[31st day of May , 1867]"


In [37]:
aldf.shape

(2136, 5)

# Post processing

**For dates**

In [38]:
G=[]
for i in aldf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
aldf["postDate-ontonotes"]=GF
            
            
        

# Run this to see the extracted dates before filtring them 

In [39]:
#GF

In [40]:
G=[]
for i in aldf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

aldf["postDate-ontonotes"]=G

**great !**

****

****

In [41]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,"] I , ] I C Meekins ] , of ] Currituck ] Count...","[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck, State of North Carolina, United St...",[Constitution]
1,"^ ] ] Department of Virginia North Carolina , ...",[],"[North Carolina, United States, States]",[OATH PAROLE]
2,"] I , ] W. H. Pearce , ] of ] Craven ] County ...","[W . H. Pearce, Almighty, God, Jm . E. Ampetts...","[Craven, County, State of North-Carolina, Unit...",[]
3,"State of North Carolina , } ________ County . ...",[God],"[State of North Carolina, United States]",[Constitution]
4,"State of North Carolina , Wake ] ] County . I ...","[S.E, Allen, God, T.D, McAlpine]","[State of North Carolina, Wake, County, United...","[Constitution, Res . Corp]"


In [42]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,"] I , ] I C Meekins ] , of ] Currituck ] Count...","[God, Isaac C. Meekins, I C Meekins E H Walker...",[Currituck ] County],[]
1,"^ ] ] Department of Virginia North Carolina , ...",[],[United States],[]
2,"] I , ] W. H. Pearce , ] of ] Craven ] County ...","[H. Pearce, God, E. Ampetts, B. Watson]","[Craven ] County, State of North-Carolina]",[]
3,"State of North Carolina , } ________ County . ...","[God, Sworn]",[United States],[]
4,"State of North Carolina , Wake ] ] County . I ...","[Allen, God, Allen Sworn]",[United States],[]


In [43]:
aldf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
aldf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
aldf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]
aldf["Person-NerEng"]=dfner["PERSON-nerenglish"]
aldf["LOC-NerEng"]=dfner["LOC-nerenglish"]
aldf["ORG-NerEng"]=dfner["ORG-nerenglish"]

In [44]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge,Person-NerEng,LOC-NerEng,ORG-NerEng
0,"] I , ] I C Meekins ] , of ] Currituck ] Count...","[I C Meekins, Isaac C. Meekins, E H Walker]",[North Carolina],[],"[16 ] day of ] Sept ] A.D . , 1865]",[16-Sept-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck, State of North Carolina, United St...",[Constitution],"[God, Isaac C. Meekins, I C Meekins E H Walker...",[Currituck ] County],[]
1,"^ ] ] Department of Virginia North Carolina , ...",[],"[North Carolina, United States]",[Department of Virginia],[1864],[],[],"[North Carolina, United States, States]",[OATH PAROLE],[],[United States],[]
2,"] I , ] W. H. Pearce , ] of ] Craven ] County ...",[W . H. Pearce],[],[],"[3rd ] day of ] October , ] A.D . , 1865]",[3-October-1865],"[W . H. Pearce, Almighty, God, Jm . E. Ampetts...","[Craven, County, State of North-Carolina, Unit...",[],"[H. Pearce, God, E. Ampetts, B. Watson]","[Craven ] County, State of North-Carolina]",[]
3,"State of North Carolina , } ________ County . ...",[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[God],"[State of North Carolina, United States]",[Constitution],"[God, Sworn]",[United States],[]
4,"State of North Carolina , Wake ] ] County . I ...",[S.E . Allen],"[North Carolina, United States]",[],"[31st day of May , 1867]",[31-May-1867],"[S.E, Allen, God, T.D, McAlpine]","[State of North Carolina, Wake, County, United...","[Constitution, Res . Corp]","[Allen, God, Allen Sworn]",[United States],[]


In [45]:
aldf.to_csv("result.csv", index=False)

In [46]:
allPerson=[]
for i in aldf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in aldf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in aldf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in aldf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in aldf["ORG-ontonotes"]:
            alORGl.append(i)

****

In [47]:
Personlarge=[]
for i in dflarge["PERSON-nerenglarge"]:
            Personlarge.append(i)
ORGlarge=[]
for i in dflarge["ORG-nerenglarge"]:
            ORGlarge.append(i) 
LOClarge=[]
for i in dflarge["LOC-nerenglarge"]:
            LOClarge.append(i)

* as usual , ner English Large is better in extracting Person 
* ontonotes is thes best in extracting dates after good postprocessing and filtring almost all pattern 
* ner Engish Large  also better in Location and Oranization 

# dates

**run those cells to take a look at the result more clearly**

In [48]:
#G

# Person

In [49]:
allPerson[:10]

[['I C Meekins', 'Isaac C. Meekins', 'E H Walker'],
 [],
 ['W . H. Pearce'],
 [],
 ['S.E . Allen'],
 ['Sarah E Dough', 'Sarah', 'E. Dough'],
 ['Sarah E. Dough', 'J.M . Wilson'],
 ['W T Dough'],
 ['Jesse E Dough',
  'Isaac C. Meekins J.P',
  'Jesse E. Dough',
  'Isaac . C. Meekins J.P'],
 ['T A Dough', 'John B Etheridge']]

In [50]:
Personlarge[:10]

[['I C Meekins',
  'Almighty',
  'God',
  'Isaac C. Meekins',
  'E H Walker',
  'J.P. John B Etheridge'],
 [],
 ['W . H. Pearce',
  'Almighty',
  'God',
  'Jm . E. Ampetts',
  'J. P.',
  'J.P. A',
  'B. Watson'],
 ['God'],
 ['S.E', 'Allen', 'God', 'T.D', 'McAlpine'],
 ['Sarah E Dough', 'God', 'Isaac C. Merkins'],
 ['Sarah E. Dough', 'J.M . Wilson'],
 ['W T Dough',
  'Almighty',
  'God',
  'W. T. Dough',
  'E H Walker',
  'J. P.',
  'J. P. John B Etheridge',
  'J. P'],
 ['Jesse E Dough',
  'God',
  'Jesse E. Dough',
  'Isaac C. Meekins J.P',
  'A. D. Etheridge'],
 ['T A Dough',
  'Almighty',
  'God',
  'Thos A Dough',
  'John B Etheridge',
  'J. P.',
  'J. P. E H Walker',
  'J. P']]

# LOC and ORG 

In [51]:
#LOClarge

In [52]:
#ORGlarge

# the final dataset :

In [53]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge,Person-NerEng,LOC-NerEng,ORG-NerEng
0,"] I , ] I C Meekins ] , of ] Currituck ] Count...","[I C Meekins, Isaac C. Meekins, E H Walker]",[North Carolina],[],"[16 ] day of ] Sept ] A.D . , 1865]",[16-Sept-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck, State of North Carolina, United St...",[Constitution],"[God, Isaac C. Meekins, I C Meekins E H Walker...",[Currituck ] County],[]
1,"^ ] ] Department of Virginia North Carolina , ...",[],"[North Carolina, United States]",[Department of Virginia],[1864],[],[],"[North Carolina, United States, States]",[OATH PAROLE],[],[United States],[]
2,"] I , ] W. H. Pearce , ] of ] Craven ] County ...",[W . H. Pearce],[],[],"[3rd ] day of ] October , ] A.D . , 1865]",[3-October-1865],"[W . H. Pearce, Almighty, God, Jm . E. Ampetts...","[Craven, County, State of North-Carolina, Unit...",[],"[H. Pearce, God, E. Ampetts, B. Watson]","[Craven ] County, State of North-Carolina]",[]
3,"State of North Carolina , } ________ County . ...",[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[God],"[State of North Carolina, United States]",[Constitution],"[God, Sworn]",[United States],[]
4,"State of North Carolina , Wake ] ] County . I ...",[S.E . Allen],"[North Carolina, United States]",[],"[31st day of May , 1867]",[31-May-1867],"[S.E, Allen, God, T.D, McAlpine]","[State of North Carolina, Wake, County, United...","[Constitution, Res . Corp]","[Allen, God, Allen Sworn]",[United States],[]


****

In [54]:
GF[26]

['Janry 15 , 1868']

In [55]:
G[26]

['15-Janry-1868']